In [80]:
def make_query(proposal_id: str) -> str:
    query = """
        {
        governorOpenZeppelinVotes(
            where: {
            proposalId: "%s"
            }
        ) {
            totalCount
            edges {
            node {
                vote_id: id
                # * platform_deployment_id #need to get from proposal
                # * proposal_id # need to get from query
                choice
                block {
                date: timestamp
                }
                voter {
                id
                }
                weight: votes
            }
            }
        }
        }
    """ % (proposal_id)
    return query

In [81]:
import requests
import backoff

@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException)
def get_query_result(proposal_id: str):
    response = requests.post('https://graphql.messari.io/query', json={'query': make_query(proposal_id)})
    data = response.json()
    return data['data']['governorOpenZeppelinVotes']['edges']

In [82]:
import pandas as pd
proposals_df = pd.read_csv('../../proposals/governor/governor_proposals.csv')
proposals_df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,2022-10-19 18:10:11,124
1,governor,fb60bedc-724e-4065-9b6f-e8ab347a7750,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,2023-03-08 18:29:11,98
2,governor,7f1ef878-12da-4848-bfe5-e9bdcee04b4a,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,2022-09-16 15:07:59,9
3,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,2023-01-23 10:48:11,413
4,governor,c34feb7f-5a4b-4ef3-9751-cb67b10a6ba2,1062472272376004331395813518426159754568225993...,0x17768cfd6030e2b65eb5086e34a512fde5dc1f1f,2023-02-21 20:14:47,3


In [83]:
proposals_df = proposals_df[['platform_deployment_id', 'proposal_id']]
proposals_df.head()

,platform_deployment_id,proposal_id
0,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110
1,fb60bedc-724e-4065-9b6f-e8ab347a7750,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49
2,7f1ef878-12da-4848-bfe5-e9bdcee04b4a,0x3d5fc645320be0a085a32885f078f7121e5e5375-26
3,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-147
4,c34feb7f-5a4b-4ef3-9751-cb67b10a6ba2,1062472272376004331395813518426159754568225993...


In [84]:
proposals_df.shape

(740, 2)

In [85]:
# remove duplicates
proposals_df = proposals_df.drop_duplicates()
proposals_df.shape

(716, 2)

In [86]:
# make dict with keys (platform_deployment_id, proposal_id) and values [votes]
votes = {}
# iterate over proposals_df
# for index, row in proposals_df.head(5).iterrows():
for index, row in proposals_df.iterrows():
    votes[(row['platform_deployment_id'], row['proposal_id'])] = []

votes

{('9a7df928-5182-40ff-9c16-8cb3f7087dbe',
  '0xec568fffba86c094cf06b22134b23074dfe2252c-110'): [],
 ('fb60bedc-724e-4065-9b6f-e8ab347a7750',
  '0xdbd27635a534a3d3169ef0498beb56fb9c937489-49'): [],
 ('7f1ef878-12da-4848-bfe5-e9bdcee04b4a',
  '0x3d5fc645320be0a085a32885f078f7121e5e5375-26'): [],
 ('9a7df928-5182-40ff-9c16-8cb3f7087dbe',
  '0xec568fffba86c094cf06b22134b23074dfe2252c-147'): [],
 ('c34feb7f-5a4b-4ef3-9751-cb67b10a6ba2',
  '106247227237600433139581351842615975456822599329363014399616832913328325135207'): [],
 ('9a7df928-5182-40ff-9c16-8cb3f7087dbe',
  '0xec568fffba86c094cf06b22134b23074dfe2252c-273'): [],
 ('1f90a7cf-17f9-4907-9f29-a7faeed44c01',
  '0xc0da02939e1441f497fd74f78ce7decb17b66529-46'): [],
 ('1f90a7cf-17f9-4907-9f29-a7faeed44c01',
  '0xc0da02939e1441f497fd74f78ce7decb17b66529-63'): [],
 ('6c9eb192-f985-4367-a6ec-e9e29363c0cd',
  '50162030097467957240858929287270422964132711578599454777185525955744917535782'): [],
 ('1f90a7cf-17f9-4907-9f29-a7faeed44c01',
  '0xc0d

In [87]:
get_query_result('00d025ec-fc9f-493c-acb9-730cd5af44f7')

[]

In [88]:
def get_governor_votes(vote_key):
    _, proposal_id = vote_key
    votes[vote_key] = get_query_result(proposal_id)
    print('done', proposal_id)

In [89]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(get_governor_votes, vote) for vote in votes.keys()]
    print(len(futures), 'futures')

    # Wait for all threads to complete
    votes_processed_list = [future.result() for future in futures]

716 futures
donedone 0x3d5fc645320be0a085a32885f078f7121e5e5375-26
 0xc0da02939e1441f497fd74f78ce7decb17b66529-63
done 0xec568fffba86c094cf06b22134b23074dfe2252c-273
done 0xc0da02939e1441f497fd74f78ce7decb17b66529-46
done 106247227237600433139581351842615975456822599329363014399616832913328325135207
done 0xdbd27635a534a3d3169ef0498beb56fb9c937489-49
done 0xec568fffba86c094cf06b22134b23074dfe2252c-110
done 0xec568fffba86c094cf06b22134b23074dfe2252c-147
done 0xec568fffba86c094cf06b22134b23074dfe2252c-227
done 0x7a6bbe7fdd793cc9ab7e0fc33605fcd2d19371e8-17
done 0xdbd27635a534a3d3169ef0498beb56fb9c937489-3
done 0xb3a87172f555ae2a2ab79be60b336d2f7d0187f0-78
done 0xc0da01a04c3f3e0be433606045bb7017a7323e38-31
done 0x95129751769f99cc39824a0793ef4933dd8bb74b-22
done 0xc0da01a04c3f3e0be433606045bb7017a7323e38-19
done 0xec568fffba86c094cf06b22134b23074dfe2252c-191
done 0x408ed6354d4973f66138c91495f2f2fcbd8724c3-39
done 0xec568fffba86c094cf06b22134b23074dfe2252c-208
done 0xb3a87172f555ae2a2ab79be60

In [90]:
votes_list = []
for key in votes.keys():
    platform_deployment_id, proposal_id = key
    proposal_votes = votes[key]
    for vote in proposal_votes:
        vote['platform_deployment_id'] = platform_deployment_id
        vote['proposal_id'] = proposal_id
        votes_list.append(vote)
len(votes_list)

412598

In [91]:
# save as json as backup...
import json
with open('./votes_backup.json', 'w') as f:
    json.dump(votes_list, f)

In [92]:
df = pd.DataFrame(votes_list)
# pull node dict out of column
df = pd.concat([df.drop(['node'], axis=1), df['node'].apply(pd.Series)], axis=1)
df.head()

,platform_deployment_id,proposal_id,vote_id,choice,block,voter,weight
0,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x070341aa5ed571f0fb2c4a5641409b1a46b4961b-110,FOR,{'date': 1666226675},{'id': '0x070341aa5ed571f0fb2c4a5641409b1a46b4...,9.504697e+22
1,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0xdc1f98682f4f8a5c6d54f345f448437b83f5e432-110,FOR,{'date': 1666385987},{'id': '0xdc1f98682f4f8a5c6d54f345f448437b83f5...,9.500005e+22
2,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x7ae109a63ff4dc852e063a673b40bed85d22e585-110,FOR,{'date': 1666420319},{'id': '0x7ae109a63ff4dc852e063a673b40bed85d22...,9.500005e+22
3,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x62a43123fe71f9764f26554b3f5017627996816a-110,FOR,{'date': 1666380755},{'id': '0x62a43123fe71f9764f26554b3f5017627996...,8.600291e+22
4,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x13bdae8c5f0fc40231f0e6a4ad70196f59138548-110,FOR,{'date': 1666309667},{'id': '0x13bdae8c5f0fc40231f0e6a4ad70196f5913...,8.500005e+22


In [93]:
# * platform
df['platform'] = 'governor'
# x platform_deployment_id
# x proposal_id
# x vote_id
# x choice -- this should be a string as this is not always true/false
# * date -- this should be as a pandas date, not as an epoch
df['date'] = df['block'].apply(lambda x: pd.to_datetime(x['date'], unit='s'))
# * voter -- the id of the person who made the vote
df['voter'] = df['voter'].apply(lambda x: x['id'])
# x weight -- the vote weight
df.head()

,platform_deployment_id,proposal_id,vote_id,choice,block,voter,weight,platform,date
0,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x070341aa5ed571f0fb2c4a5641409b1a46b4961b-110,FOR,{'date': 1666226675},0x070341aa5ed571f0fb2c4a5641409b1a46b4961b,9.504697e+22,governor,2022-10-20 00:44:35
1,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0xdc1f98682f4f8a5c6d54f345f448437b83f5e432-110,FOR,{'date': 1666385987},0xdc1f98682f4f8a5c6d54f345f448437b83f5e432,9.500005e+22,governor,2022-10-21 20:59:47
2,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x7ae109a63ff4dc852e063a673b40bed85d22e585-110,FOR,{'date': 1666420319},0x7ae109a63ff4dc852e063a673b40bed85d22e585,9.500005e+22,governor,2022-10-22 06:31:59
3,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x62a43123fe71f9764f26554b3f5017627996816a-110,FOR,{'date': 1666380755},0x62a43123fe71f9764f26554b3f5017627996816a,8.600291e+22,governor,2022-10-21 19:32:35
4,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x13bdae8c5f0fc40231f0e6a4ad70196f59138548-110,FOR,{'date': 1666309667},0x13bdae8c5f0fc40231f0e6a4ad70196f59138548,8.500005e+22,governor,2022-10-20 23:47:47


In [94]:
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'vote_id', 'voter', 'date', 'choice', 'weight']]
df.head()

,platform,platform_deployment_id,proposal_id,vote_id,voter,date,choice,weight
0,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x070341aa5ed571f0fb2c4a5641409b1a46b4961b-110,0x070341aa5ed571f0fb2c4a5641409b1a46b4961b,2022-10-20 00:44:35,FOR,9.504697e+22
1,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0xdc1f98682f4f8a5c6d54f345f448437b83f5e432-110,0xdc1f98682f4f8a5c6d54f345f448437b83f5e432,2022-10-21 20:59:47,FOR,9.500005e+22
2,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x7ae109a63ff4dc852e063a673b40bed85d22e585-110,0x7ae109a63ff4dc852e063a673b40bed85d22e585,2022-10-22 06:31:59,FOR,9.500005e+22
3,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x62a43123fe71f9764f26554b3f5017627996816a-110,0x62a43123fe71f9764f26554b3f5017627996816a,2022-10-21 19:32:35,FOR,8.600291e+22
4,governor,9a7df928-5182-40ff-9c16-8cb3f7087dbe,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x13bdae8c5f0fc40231f0e6a4ad70196f59138548-110,0x13bdae8c5f0fc40231f0e6a4ad70196f59138548,2022-10-20 23:47:47,FOR,8.500005e+22


In [95]:
df.shape

(412598, 8)

In [96]:
# save as governor_votes.csv
df.to_csv('./governor_votes.csv', index=False)